In [ ]:
import csv
import json
import time
import datetime
import math
from abc import ABC, abstractmethod

"""Class to create a person from user input"""
class Person:
    def __init__(self, first_name:str, last_name:str, gender:str, email:str):
        self.first_name=first_name
        self.last_name=last_name
        self.gender=gender
        self.email=email
        
    def __str__(self):
        return "Hello you have applied for membership({} {} {} {})".format(self.first_name, self.last_name,self.gender, self.email)

"""Class to create a new member with member id and membership number"""   
class Create_New_Member(Person):
    
    def __init__(self,member_id:int,first_name:str,last_name:str,gender:str,email:str,membership_number:int):
        Person.__init__(self, first_name,last_name, gender, email)
        self.member_id=member_id
        self.membership_number=membership_number
        
        #Adds the new member data into a dictionary for later
        self.new_member_dict={"Member ID": self.member_id, "First Name":self.first_name, 
                                "Last name": self.last_name,"Gender": self.gender,"Email": self.email,
                                 "Membership Number": self.membership_number}  
    
        
    """Function to convert member data to json"""
    def convert_dictionary_to_json(self,json_file):
        with open(json_file, mode='w') as file:
            
            #add the member information from dictionary into a json file
            #and return the json
            book_json=json.dump(self.new_member_dict,file)
            return book_json

"""The Member class"""
class Member:
    
    def scan_member_number(self, member_no):
        return member_no
        
"""The Book class"""
class Book:
    
    def scan_book_id(self,book_id):
        return book_id
    

"""Library Management class performs generic helper function that can be called for other classes"""
class Library_Management:

        
    """Get a subset of data from the an inputted csv file"""    
    def read_csv_file(self, file_name):
        data=[]
        rowCount=0
        
        with open(file_name,mode="r", encoding="utf-8-sig") as file:
            content=csv.reader(file)
            
            #get the all row data
            for row in content:
                if rowCount==0:
                    heading=row
                    rowCount+=1
                else:
                    data+=[row]
        return data
        
        
    """Function to retrieve the max number of values in array and create a new ID based on the current values"""
    def create_new_id (self, inputted_array=[]):
        array_range=(range(len(inputted_array)+1))
        new_value=(max(array_range)+1)
        return new_value
    
    """Function to create new member number"""
    def create_new_member_number (self, inputted_array=[]):
        array_range=(range(len(inputted_array)+1))
        new_value=(str(max(array_range)+1)+'1')
        return new_value

    """Function to update JSON file data"""
    def update_json_file(self,key,value,file_name):
        with open(file_name, mode="r+")as file:
            json_data = json.load(file)
            json_data[key] = value
            file.seek(0)
            file.write(json.dumps(json_data))
            file.truncate()
            
    """Function to create a json file"""
    def create_json_file(self, new_dict,json_file):
        with open(json_file, mode='w') as file:
            
            book_json=json.dump(new_dict,file)
            return book_json
    
    """Function to read JSON file"""
    def read_json_file(self,file_name):
        with open(file_name, mode="r")as file:
            json_data = json.load(file)
            return json_data
        
    """Function to convert array of strings into integers"""
    def convert_string_to_integer(self,inputted_array):
        int_convert=[int(val) for val in inputted_array]
        return int_convert
    
    """Function to format the current date based on the unix epoch time """
    def format_date(self,refVal,dateVal):
        formatRef=datetime.datetime.strptime(refVal, "%Y-%m-%d")
        newDate=(formatRef+datetime.timedelta(days=int(dateVal))).strftime("%Y-%m-%d")
        return newDate
    
    """Function to update dictionary with new key value"""
    def update_dictionary(self,update_dict,key, value):
        
        return update_dict.update({key:value})
        
library_system=Library_Management()

"""Abstract Library Method class """
class Library_Methods(ABC):
    @abstractmethod
    def book_methods():
        pass

"""Class to borrow book"""
"""Pre Condition"""
#All fields must be filled out (not empty)
#Numbers must be input for book_id and member_id
#Email must be in a valid format e.g daniel@gmail.com
"""Post Conditions"""
#Must create a json file which looks like the below as an example - borrow_new_book.json
#{"Book ID": 13, "Member ID": 13, "Date Borrowed": "2022-01-03", "Date Returned": 0}
class Borrow_Book(Library_Methods):
    
    def book_methods(self):
        #Input ID's and call the scan methods
        book_id=int(input("Scan Book ID"))
        inputted_book_id=Book()
        scanned_book_id=inputted_book_id.scan_book_id(book_id)
        print(scanned_book_id)
        
        member_id=int(input("Scan Membership Card"))
        inputted_member_id=Member()
        scanned_member_id=inputted_member_id.scan_member_number(member_id)
        print(scanned_member_id)
        
    
        #Check the ID's exists in the csv files
        book_id_data=library_system.read_csv_file('books.csv')
        member_id_data=library_system.read_csv_file('members.csv')
        book_id_arr=[]
        member_id_arr=[]
    
        for row in book_id_data:
            book_id_arr+=[row[0]]
        
        for row in member_id_data:
            member_id_arr+=[row[5]]
            
        #Convert array values to integers
        convert_book_id=library_system.convert_string_to_integer(book_id_arr)
        convert_member_id=library_system.convert_string_to_integer(member_id_arr)
        
        loan_dict={}
        #Convert epoch time to current date"""
        seconds_to_days=(math.trunc(time.time())/86400)
        round_days=(math.floor(seconds_to_days))
        current_date=library_system.format_date("1970-01-01",round_days)
        
        #check if values exist and if so create json file to hold borowed book details
        if int(scanned_book_id) in convert_book_id and int(scanned_member_id) in convert_member_id:
            loan_dict={"Book ID": scanned_book_id, "Member ID": scanned_member_id, "Date Borrowed":current_date, "Date Returned": 0}
            library_system.create_json_file(loan_dict,'borrow_new_book.json')
            print("book borrowed :)")
        else:
            print("book not borrowed :(")
        
"""Class to return book"""
"""Pre Condition"""
#Numbers must be input for book_id and member_id
#borrow_new_book.json must already exist so it can updated
"""Post Conditions"""
#Must update the current json file so returned date is added and a late return field is set to yes
#{"Book ID": 13, "Member ID": 13, "Date Borrowed": "2022-01-03", "Date Returned": "2022-01-03", "Late Return": "Yes"}   
class Return_Book(Library_Methods):
    def book_methods(self):
       
        #Input ID's and call the scan methods
        book_id=int(input("Scan Book ID"))
        inputted_book_id=Book()
        scanned_book_id=inputted_book_id.scan_book_id(book_id)
        print(scanned_book_id)
        
        member_id=int(input("Scan card number"))
        inputted_member_id=Member()
        scanned_member_id=inputted_member_id.scan_member_number(member_id)
        print(scanned_member_id)
    
        #Return the json file as a readable dictionary"""
        read_file=library_system.read_json_file('borrow_new_book.json')

        #Call methods to get the book id and member id data
        book_id_data=library_system.read_csv_file('books.csv')
        member_id_data=library_system.read_csv_file('members.csv')
        book_id_arr=[]
        member_id_arr=[]
    
        for row in book_id_data:
            book_id_arr+=[row[0]]
        
        for row in member_id_data:
            member_id_arr+=[row[5]]
        
        #Convert epoch to current date
        seconds_to_days=(math.trunc(time.time())/86400)
        round_days=(math.floor(seconds_to_days))
        current_date=library_system.format_date("1970-01-01",round_days)
        #print(current_date)
        
        #Convert array values to integers
        convert_book_id=library_system.convert_string_to_integer(book_id_arr)
        convert_member_id=library_system.convert_string_to_integer(member_id_arr)
    
        #Check If values exist in the dictionary and if update borrow_new_book json with the returned date
        if int(scanned_book_id) in convert_book_id and int(scanned_member_id) in convert_member_id:
            library_system.update_json_file("Date Returned",current_date,'borrow_new_book.json')
            library_system.update_json_file("Late Return","Yes",'borrow_new_book.json')
            print("succesfully returned book")
        else:
            print("not returned :(")
       
        
    

"""Class to apply for membership book"""
"""Pre Condition"""
#Email format must be added correctly
#gender must be Male or Female
"""Post Conditions"""
#Must create a new member json file which looks like the below
#{"Member ID": 201, "First Name": "daniel", "Last name": "punter", "Gender": "Male", "Email": "daniel@gmail.com", "Membership Number": 0}   
class Apply_For_Membership(Library_Methods):
    def book_methods(self):
        #Input ID's and call the scan methods
        new_fname=input("enter your first name")
        new_lname=input("enter your surname")
        new_gender=input("enter gender")
        new_email=input("enter your email")
        new_person=Person(new_fname, new_lname, new_gender, new_email)
        print(new_person)
    
        #Call the read csv file to read data
        member_id_data=library_system.read_csv_file('members.csv')
        member_id=[]
        for row in member_id_data:
            member_id+=[row[0]]
            
        #Create a new ID for the user, succesfully outputs 201 as the ID
        new_member_id=library_system.create_new_id(member_id)
        #print(new_member_id)
    
        #Create Member with the ID
        create_member=Create_New_Member(new_member_id, new_fname, new_lname, new_gender, new_email,0)
    
        #Call Method to create new json file with new member details
        create_member.convert_dictionary_to_json('new_member.json')
        print("Hello", new_fname, new_lname," you successfully applied for membership")
  
"""Update membership number class"""
"""Pre Condition"""
#new_member.json must exist
"""Post Conditions"""
#Must update the current json file with a membership number which looks like the below
#{"Member ID": 201, "First Name": "daniel", "Last name": "punter", "Gender": "Male", "Email": "daniel@gmail.com", "Membership Number": 2011}   
class Update_Membership_Number(Library_Methods):

    def book_methods(self):
        #get member id data and find the max value then add 1
        member_id_data=library_system.read_csv_file('members.csv')
        member_id=[]
        for row in member_id_data:
            member_id+=[row[0]]

        new_membership_number=library_system.create_new_member_number(member_id)
        #update json file to update the record with the new card number
        library_system.update_json_file("Membership Number",int(new_membership_number),'new_member.json')
        print("membership number sucessfully updated")

"""Reserve book class"""
"""Pre Condition"""
#inputted id's must be numbers
"""Post Conditions"""
#Must create a reserved book json json file which looks like the below
#{"Title": "Fundamentals of Wavelets", "Author": "Goswami, Jaideva", "Genre": "tech", "Sub Genre": "signal_processing",
#"Publisher": "Wiley", "Book ID": 1, "Member ID": 13, "Status": "Available"}

class Reserve_Book(Library_Methods):
    def book_methods(self):
        #Input IDs and call the scan method
        book_id=int(input("Scan Book ID"))
        inputted_book_id=Book()
        scanned_book_id=inputted_book_id.scan_book_id(book_id)
        print("Book ID", scanned_book_id)
        
        member_id=int(input("Scan card number"))
        inputted_member_id=Member()
        scanned_member_id=inputted_member_id.scan_member_number(member_id)
        print("Member ID",scanned_member_id)
    
        #Get the existing book id, data and card number from the csv files"""
        book_id_data=library_system.read_csv_file('books.csv')
        member_id_data=library_system.read_csv_file('members.csv')
    
        book_id_arr=[]
        title=[]
        author=[]
        genre=[]
        sub_genre=[]
        publisher=[]
        member_id_arr=[]

        #book dictionary to hold all the book data
        book_dict={}
        udpated_book_dict={}
        for row in book_id_data:
            book_dict.update({int(row[0]):{"Title":(row[1]),"Author": (row[2]),
                "Genre": (row[3]),"Sub Genre":(row[4]),"Publisher": (row[5])}})
    
            book_id_arr+=[row[0]]
    
        for row in member_id_data:
            member_id_arr+=[row[5]]

        convert_book_id=library_system.convert_string_to_integer(book_id_arr)
        convert_member_id=library_system.convert_string_to_integer(member_id_arr)
   
        #Return the book info if book id and member id matches"""
        if int(book_id) in convert_book_id and int(member_id) in convert_member_id:
            #Put book details into a dictionary
            updated_book_dict=book_dict[book_id]
        
            #Add the book id and member id details and status into a new dictionary for reserving the book
            library_system.update_dictionary(updated_book_dict,"Book ID",book_id)
            library_system.update_dictionary(updated_book_dict,"Member ID",member_id)
            library_system.update_dictionary(updated_book_dict,"Status","Available")
        
            #update book into reserved book json file"""
            library_system.create_json_file(updated_book_dict,'reserved_book.json')
            
            print("book succesfully reserved :)")
        else:
            print("book not reserved :(")
    
      
class Notification(ABC):
    @abstractmethod
    def notification_email():
        pass

"""Class to check if reserved book is available"""
class Reserved_Book_Notification(Notification):
    
    def notification_email(self):
        """Scan  user input"""
        member_id=int(input("Scan card number"))
        inputted_member_id=Member()
        scanned_member_id=inputted_member_id.scan_member_number(member_id)
        print("Member ID",scanned_member_id)
      
        file=library_system.read_json_file('reserved_book.json')
        available_status=file["Status"]
        available_member=file["Member ID"]
        
        if int(scanned_member_id)==available_member:
            if available_status == "Available":
                print("available")
            else:
                print("not available or does not match your member id")
    
"""Class to check if ordered book is available"""
class Ordered_Book_Notification(Notification):
    
    def notification_email(self):
        """Scan  user input"""
        member_id=int(input("Scan card number"))
        inputted_member_id=Member()
        scanned_member_id=inputted_member_id.scan_member_number(member_id)
        print("Member ID",scanned_member_id)
        
        retrieve_json=library_system.read_json_file('ordered_books.json')
        available_status=retrieve_json["Available"]
        available_member=retrieve_json["Member ID"]
        
        if int(scanned_member_id) == int(available_member):
            if available_status == "Yes":
                print("the ordered book is available")
            else:
                print("the ordered book is not available")

"""Class to check if there are any outstanding fines"""
class Fine_Notification(Notification):
    
    def notification_email(self):
        """Scan  user input"""
        member_id=int(input("Scan card number"))
        inputted_member_id=Member()
        scanned_member_id=inputted_member_id.scan_member_number(member_id)
        print("Member ID",scanned_member_id)
        
        file=library_system.read_json_file('borrow_new_book.json')
        fine_status=file["Late Return"]
        available_member=file["Member ID"]
        
        if int(scanned_member_id) == int(available_member):
            if fine_status != "Yes":
                print("No outstanding fines")
            else:
                print("Yes you have an outstanding book that is overdue you will be fined £20")

"""Static method which takes user input and returns a library method"""
class Library_System:
    @staticmethod
    def library_options(choice):
        if int(choice)==1:
            return Borrow_Book()
        elif int(choice)==2:
            return Return_Book()
        elif int(choice)==3:
            return Apply_For_Membership()
        elif int(choice)==4:
            return Update_Membership_Number()
        elif int(choice)==5:
            return Reserve_Book()
        else:
            return None
        
"""Static method which takes user input and returns a notification method"""
class Create_Notification:
    @staticmethod
    def new_notification(choice):
        if int(choice)==1:
            return Reserved_Book_Notification()
        elif int(choice)==2:
            return Ordered_Book_Notification()
        elif int(choice)==3:
            return Fine_Notification()
        else:
            return None
        
"""Function to call library methods"""
def lib_system():
    user_choice=(input("Choose one of the following options\n1) Borrow a Book\n2) Return Book\n3)Apply For Membership\n4)Update Membership Number\n5)Reserve Book"))
    lib=Library_System.library_options(user_choice)
    lib.book_methods()

lib_system()

"""Function to call notification methods"""
def notification_system():
    user_choice=(input("Choose one of the following options\n1)Check if reserved book is available\n2)Check if ordered book is available\n3)Check if any outstanding fines exist"))
    notify=Create_Notification.new_notification(user_choice)
    notify.notification_email()
    
#notification_system()

## 